In [2]:
#import the dataset from pycaret repository 
from pycaret.datasets import get_data 
import pandas as pd
# boston = get_data('boston') 

df = pd.read_pickle("./full_data_no_missing_combine_selected.pkl")

#import regression module 
from pycaret.regression import * 

#intialize the setup (in Notebook env)
# exp_reg = setup(df, target = 'medv')

#intialize the setup (in Non-Notebook env)
# exp_reg = setup(boston, target = 'medv', html = False)

#intialize the setup (remote runs like Kaggle or GitHub actions)
# exp_reg = setup(boston, target = 'medv', silent = True)

In [ ]:
# %tb
train_data = df[:1460]
numeric_features=[

                  'GarageFinish',
                'FullBath',
                'BsmtExposure',
                'ExterQual',
                  '1stFlrSF',
                  'GarageScale',
                  'OpenPorchSF',
                  'GarageArea',
                  'GarageYrBlt',
                  'MasVnrArea',
                  '2ndFlrSF',
                  'BsmtFinSF1',
                  'YearRemodAdd',
                  'TotalFlrSF',
                  'TotalFullBath',
                  'WoodDeckSF',
                  'TotalBsmtSF',
                  'TotRmsAbvGrd',
                  'GrLivArea',
                  'GarageCars',
                  'LotFrontage',
                  'YearBuilt',
                  'TotalSF',

              ]

# ordinal_features_names = ['KitchenQual',
#  'BsmtFinType1',
#  'BsmtQual',
#  'OverallExterScore',
#  'FireplaceQu',
#  'OverallQual',
#  'OverallAbvGrKitchenScore',
#  'OverallHouseScore',
#  'Fireplaces',
#  'GarageFinish',
#  'FullBath',
#  'BsmtExposure',
#  'ExterQual']

# ordinal_features = {f:list(set(train_data[f].values)) for f in ordinal_features_names}

# for k,v in ordinal_features.items():
#     sorted(v)
# print(ordinal_features)

categorical_features=[
                 'Neighborhood_NoRidge',
                 'Foundation_PConc',
                 'GarageType_Detchd',
                 'HeatingQC_Ex',
                 'Neighborhood_NridgHt',
                 'GarageType_Attchd',
                 'Foundation_CBlock',
                 'MasVnrType_None',
                 'SaleType_New',
                 'HeatingQC_TA',
                 'MasVnrType_3',
                 'Exterior1st_VinylSd',
                 'SaleCondition_Partial',
                 'Exterior2nd_VinylSd',
             ]

exp = setup(data = train_data, target = 'SalePrice', session_id=8891,
              ignore_features=['Id'],
             categorical_features=categorical_features,
#                 ordinal_features=ordinal_features,
              numeric_features=numeric_features,
             )

In [3]:
import numpy as np
train_data = df[:1460]
train_data.SalePrice = np.log(train_data.SalePrice)
col_names = list(set(train_data.columns))
print(train_data['SalePrice'].head())
col_names.remove('SalePrice')
numeric_features = [c for c in col_names if '_' not in c]
categorical_features =  [c for c in col_names if '_' in c]

#8891->brent

# exp = setup(data = train_data, target = 'SalePrice', # session_id=8891,
#               ignore_features=['Id'],
#              categorical_features=categorical_features,
# #                 ordinal_features=ordinal_features,
#               numeric_features=numeric_features,
#             train_size=0.7
#              )
# 5946
exp = setup(data = train_data, target = 'SalePrice',
              ignore_features=['Id'],
#              categorical_features=categorical_features,
#                 ordinal_features=ordinal_features,
              numeric_features=list(col_names),
            train_size=0.8
             )

 
Setup Succesfully Completed.


,Description,Value
0,session_id,6425
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(1460, 74)"
4,Missing Values,False
5,Numeric Features,73
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [4]:
# best = compare_models(sort = 'RMSE', n_select = 5, fold=3) #default is 'R2'
model = create_model('catboost', fold = 3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0854,0.0150,0.1226,0.9101,0.0095,0.0071
1,0.0848,0.0147,0.1214,0.8975,0.0094,0.0071
2,0.0926,0.0214,0.1464,0.8805,0.0113,0.0078
Mean,0.0876,0.0171,0.1301,0.8961,0.0101,0.0073
SD,0.0036,0.0031,0.0115,0.0121,0.0009,0.0003


In [5]:
tuned_catboost = tune_model(model, optimize = 'RMSE', fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0822,0.0138,0.1175,0.9077,0.0089,0.0068
1,0.0869,0.0163,0.1276,0.9062,0.0101,0.0074
2,0.0827,0.0145,0.1204,0.9040,0.0094,0.0069
3,0.0906,0.0161,0.1269,0.9006,0.0099,0.0076
4,0.0906,0.0232,0.1523,0.8691,0.0117,0.0076
Mean,0.0866,0.0168,0.1290,0.8975,0.0100,0.0072
SD,0.0037,0.0033,0.0123,0.0144,0.0010,0.0003


In [7]:
buit_model = tuned_catboost
prediction = predict_model(buit_model, data = df[1460:])
submission = prediction[['Id', 'Label']]
submission = submission.rename(columns={"Label": "SalePrice"})
submission.SalePrice = np.exp(submission.SalePrice)
submission.to_csv('submission_catboost_default_settings_0.8_num_only.csv', index=False)

In [5]:
top3 = compare_models(n_select = 3, fold=3, sort = 'RMSE')
tuned_top3 = [tune_model(i, optimize='RMSE') for i in top3]
buit_model = stack_models(estimator_list = tuned_top3[1:], meta_model = tuned_top3[0])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.1052,0.0219,0.1479,0.8797,0.0113,0.0087
1,0.0783,0.0109,0.1045,0.9207,0.0080,0.0065
2,0.1027,0.0292,0.1709,0.8430,0.0133,0.0087
3,0.0913,0.0208,0.1443,0.8636,0.0114,0.0077
4,0.0802,0.0150,0.1226,0.9152,0.0097,0.0068
5,0.0924,0.0215,0.1467,0.8757,0.0111,0.0077
6,0.0752,0.0102,0.1011,0.9276,0.0078,0.0063
7,0.0890,0.0170,0.1305,0.9007,0.0100,0.0074
8,0.0915,0.0162,0.1274,0.8987,0.0099,0.0077
9,0.0773,0.0122,0.1105,0.9304,0.0085,0.0064


In [11]:
# display(testing_data.info())
# display(submission.info())

prediction = predict_model(buit_model, data = df[1460:])
submission = prediction[['Id', 'Label']]
submission = submission.rename(columns={"Label": "SalePrice"})
submission.SalePrice = np.exp(submission.SalePrice)
submission.to_csv('submission_top3_stacked_default_settings.csv', index=False)